In [ ]:
#!pip install --user -qq datasets==2.20.0 trl==0.9.6 transformers==4.42.3 peft==0.11.1 tqdm==4.66.4 numpy==1.26.4 pandas==2.2.2 matplotlib==3.9.1 seaborn==0.13.2 scikit-learn==1.5.1 sacrebleu==2.4.2 evaluate==0.4.2

In [ ]:
#!pip install --upgrade evaluate

In [ ]:
#!pip install --upgrade transformers

In [ ]:
#!pip install --upgrade huggingface_hub

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install trl==0.9.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
import torch
from torch.utils.data import Dataset
from tqdm import tqdm
import evaluate
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

from peft import get_peft_model, LoraConfig, TaskType

import pickle
import json
import matplotlib.pyplot as plt

from urllib.request import urlopen
import io

In [ ]:
#write your code here
qaDataset = load_dataset("rajpurkar/squad")
qaDataset = qaDataset.shuffle(seed = 42)

In [ ]:
qaDataset["train"][999]

{'id': '57318409e6313a140071cffc',
 'title': 'Indigenous_peoples_of_the_Americas',
 'context': "Approximately 96.4% of Ecuador's Indigenous population are Highland Quichuas living in the valleys of the Sierra region. Primarily consisting of the descendents of Incans, they are Kichwa speakers and include the Caranqui, the Otavalos, the Cayambi, the Quitu-Caras, the Panzaleo, the Chimbuelo, the Salasacan, the Tugua, the Puruhá, the Cañari, and the Saraguro. Linguistic evidence suggests that the Salascan and the Saraguro may have been the descendants of Bolivian ethnic groups transplanted to Ecuador as mitimaes.",
 'question': 'The Saraguro and Panzaleo are two of the member groups of what population?',
 'answers': {'text': ['Highland Quichuas'], 'answer_start': [59]}}

In [ ]:
qaDataset.filter(lambda x: x['answers']['text'] == "")

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 0
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 0
    })
})

In [ ]:
trainQADS = qaDataset["train"].filter(lambda x: x['answers']['text'] != "")
testQADS = qaDataset["validation"].filter(lambda x: x['answers']['text'] != "")

In [ ]:
trainQADS

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [ ]:
len(trainQADS['question'])

87599

In [ ]:
trainQADS['answers'][0]['text'][0]

'84%'

In [ ]:
def filterByLength(x):
    text = (
            f"### Context:\n{x['context']} \n"
            f"### Question:\n{x['question']} \n"
            f"\n\n### Answer:\n{x['answers']['text'][0]}</s>"
        )
    if len(text) > 2048:
        return False
    return True

In [ ]:
trainQADS = trainQADS.filter(filterByLength)

In [ ]:
len(trainQADS)

87132

In [ ]:
testQADS = testQADS.filter(filterByLength)

In [ ]:
def formattQaDSPrompt(qaDS):
    output_texts = []
    # Iterate over each example in the batch
    for i in range(len(qaDS['question'])):
        text = (
            f"### Context:\n{qaDS['context'][i]} \n"
            f"### Question:\n{qaDS['question'][i]} \n"
            f"\n\n### Answer:\n{qaDS['answers'][i]['text'][0]}</s>"
        )

        output_texts.append(text)
    return output_texts

In [ ]:
trainDS = formattQaDSPrompt(trainQADS)

In [ ]:
trainDS[0]

'### Context:\nThe Pew Forum on Religion & Public Life ranks Egypt as the fifth worst country in the world for religious freedom. The United States Commission on International Religious Freedom, a bipartisan independent agency of the US government, has placed Egypt on its watch list of countries that require close monitoring due to the nature and extent of violations of religious freedom engaged in or tolerated by the government. According to a 2010 Pew Global Attitudes survey, 84% of Egyptians polled supported the death penalty for those who leave Islam; 77% supported whippings and cutting off of hands for theft and robbery; and 82% support stoning a person who commits adultery. \n### Question:\nWhat percentage of Egyptians polled support death penalty for those leaving Islam? \n\n\n### Answer:\n84%</s>'

In [ ]:
def formattQaDSPromptNoResponse(qaDS):
    output_texts = []
    # Iterate over each example in the batch
    for i in range(len(qaDS['question'])):
        text = (
            f"### Context:\n{qaDS['context'][i]} \n"
            f"### Question:\n{qaDS['question'][i]}\n"
            f"\n\n### Answer:\n"
        )
        output_texts.append(text)
    return output_texts

In [ ]:
testDS = formattQaDSPromptNoResponse(testQADS)

In [ ]:
testDSWithAnswer = formattQaDSPrompt(testQADS)

In [ ]:
trainDSNoAnswer = formattQaDSPromptNoResponse(trainQADS)

In [ ]:
class ListDataset(Dataset):
    def __init__(self, original_list):
        super(ListDataset, self).__init__()
        self.original_list = original_list

    def __len__(self):
        return len(self.original_list)

    def __getitem__(self, i):
        return self.original_list[i]

In [ ]:
#processedTrainQADS = ListDataset([x for x in trainDS if len(x) <= 2048])
processedTrainQADS = ListDataset([x for x in trainDS])

In [ ]:
#processedTrainQADSNoAnswer = ListDataset([x for x in trainDSNoAnswer if len(x) <= 2048])
processedTrainQADSNoAnswer = ListDataset([x for x in trainDSNoAnswer])

In [ ]:
processedTestQADS = ListDataset([x for x in testDSWithAnswer])

In [ ]:
len(processedTrainQADS)

87132

In [ ]:
len(processedTrainQADSNoAnswer)

87132

In [ ]:
processedTrainQADS[18]

"### Context:\nDisease can arise if the host's protective immune mechanisms are compromised and the organism inflicts damage on the host. Microorganisms can cause tissue damage by releasing a variety of toxins or destructive enzymes. For example, Clostridium tetani releases a toxin that paralyzes muscles, and staphylococcus releases toxins that produce shock and sepsis. Not all infectious agents cause disease in all hosts. For example, less than 5% of individuals infected with polio develop disease. On the other hand, some infectious agents are highly virulent. The prion causing mad cow disease and Creutzfeldt–Jakob disease invariably kills all animals and people that are infected. \n### Question:\nWhat does the of toxin Clostridium tetani releases do? \n\n\n### Answer:\nparalyzes muscles</s>"

In [ ]:
processedTrainQADSNoAnswer[18]

"### Context:\nDisease can arise if the host's protective immune mechanisms are compromised and the organism inflicts damage on the host. Microorganisms can cause tissue damage by releasing a variety of toxins or destructive enzymes. For example, Clostridium tetani releases a toxin that paralyzes muscles, and staphylococcus releases toxins that produce shock and sepsis. Not all infectious agents cause disease in all hosts. For example, less than 5% of individuals infected with polio develop disease. On the other hand, some infectious agents are highly virulent. The prion causing mad cow disease and Creutzfeldt–Jakob disease invariably kills all animals and people that are infected. \n### Question:\nWhat does the of toxin Clostridium tetani releases do?\n\n\n### Answer:\n"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
qaModel = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m").to(device)

In [ ]:
qaTokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m", padding_side='left')

In [ ]:
expected_outputs = []

tokenizedQuestionWithAnswer = qaTokenizer(processedTrainQADS[0], return_tensors="pt", max_length=1024, truncation=True, padding=False)
tokenizedQuestion = qaTokenizer(processedTrainQADSNoAnswer[0], return_tensors="pt", max_length=1024, truncation=True, padding=False)
expectedAnswer = qaTokenizer.decode(tokenizedQuestionWithAnswer['input_ids'][0][len(tokenizedQuestion['input_ids'][0]) + 1:], skip_special_tokens=True)
basePart = qaTokenizer.decode(tokenizedQuestionWithAnswer['input_ids'][0][:len(tokenizedQuestion['input_ids'][0])], skip_special_tokens=True)
expected_outputs.append(expectedAnswer)

NameError: name 'expected_outputs' is not defined

In [ ]:
processedTrainQADS[0]

In [ ]:
expectedAnswer

In [ ]:
expected_outputs = []

for i in tqdm(range(len(processedTrainQADS))):
    tokenizedQuestionWithAnswer = qaTokenizer(processedTrainQADS[i], return_tensors="pt", max_length=1024, truncation=True, padding=False)
    tokenizedQuestion = qaTokenizer(processedTrainQADSNoAnswer[i], return_tensors="pt")
    expectedAnswer = qaTokenizer.decode(tokenizedQuestionWithAnswer['input_ids'][0][len(tokenizedQuestion['input_ids'][0])+1:], skip_special_tokens=True)
    expected_outputs.append(expectedAnswer)

100%|██████████| 87132/87132 [02:05<00:00, 695.07it/s]


In [ ]:
processedTrainQADS[88]

"### Context:\nMost of Bermuda's black population trace some of their ancestry to Native Americans, although awareness of this is largely limited to St David's Islanders and most who have such ancestry are unaware of it. During the colonial period, hundreds of Native Americans were shipped to Bermuda. The best-known examples were the Algonquian peoples who were exiled from the southern New England colonies and sold into slavery in the 17th century, notably in the aftermaths of the Pequot and King Philip's wars. \n### Question:\nWhat is one group that Bermuda's black population can link some of their ancestry to? \n\n\n### Answer:\nNative Americans</s>"

In [ ]:
expected_outputs[88]

'Native Americans</s>'

In [ ]:
qaPipeline = pipeline("text-generation",
                        model = qaModel,
                        tokenizer = qaTokenizer,
                        device=device,
                        batch_size=16,
                        truncation=True,
                        max_new_tokens=50,
                        padding = True,
                        return_full_text = False)

Device set to use cuda


In [ ]:
qaPipeline.tokenizer.pad_token_id = qaModel.config.eos_token_id

In [ ]:
batchNum = 100

In [ ]:
with torch.no_grad():
  qaGeneratedAnswersIter = qaPipeline(processedTrainQADSNoAnswer[:batchNum],
                                #max_length=50, # this is set to 50 due to resource constraint, using a GPU, you can increase it to the length of your choice
                                #num_beams=5,
                                #early_stopping=True,
                                      )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
len(qaGeneratedAnswersIter)

100

In [ ]:
len(processedTrainQADS[:batchNum])

100

In [ ]:
generated_outputs_base = []
for i, text in enumerate(qaGeneratedAnswersIter):
    generated_outputs_base.append(text[0]["generated_text"])
    #print(f"Question: {processedTrainQADSNoAnswer[i]}")
    #print(f"Generated Answer: {text[0]['generated_text']}")len()


sacrebleu = evaluate.load("sacrebleu")

results_base = sacrebleu.compute(predictions=generated_outputs_base,
                                 references=expected_outputs[:batchNum])

print(list(results_base.keys()))
print(round(results_base["score"], 1))

['score', 'counts', 'totals', 'precisions', 'bp', 'sys_len', 'ref_len']
0.4


In [ ]:
generated_outputs_base[9]

'In Australia it is often the Aborigines who carry the symbolic significance of the symbolic character given to them by the animal (e.g. the white rabbit).\n\n### Context:\nAustralia is a large, temperate country, with a'

In [ ]:
expected_outputs[9]

'honey ants</s>'

In [ ]:
trainDSNoAnswer[9]

"### Context:\nScarab beetles held religious and cultural symbolism in Old Egypt, Greece and some shamanistic Old World cultures. The ancient Chinese regarded cicadas as symbols of rebirth or immortality. In Mesopotamian literature, the epic poem of Gilgamesh has allusions to Odonata which signify the impossibility of immortality. Amongst the Aborigines of Australia of the Arrernte language groups, honey ants and witchety grubs served as personal clan totems. In the case of the 'San' bush-men of the Kalahari, it is the praying mantis which holds much cultural significance including creation and zen-like patience in waiting.:9 \n### Question:\nWhat kind of ants are symbolic among the Australian Aborigines?\n\n\n### Answer:\n"

In [ ]:
trainDS[9]

"### Context:\nScarab beetles held religious and cultural symbolism in Old Egypt, Greece and some shamanistic Old World cultures. The ancient Chinese regarded cicadas as symbols of rebirth or immortality. In Mesopotamian literature, the epic poem of Gilgamesh has allusions to Odonata which signify the impossibility of immortality. Amongst the Aborigines of Australia of the Arrernte language groups, honey ants and witchety grubs served as personal clan totems. In the case of the 'San' bush-men of the Kalahari, it is the praying mantis which holds much cultural significance including creation and zen-like patience in waiting.:9 \n### Question:\nWhat kind of ants are symbolic among the Australian Aborigines? \n\n\n### Answer:\nhoney ants</s>"

# Data Collator Setup
Example usage: https://gist.github.com/Blaizzy/40de0f6b4340490e3920db9e182e6455

In [ ]:
response_template = "### Answer:\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=qaTokenizer)

In [ ]:
processedTrainQADS[0]

'### Context:\nThe Pew Forum on Religion & Public Life ranks Egypt as the fifth worst country in the world for religious freedom. The United States Commission on International Religious Freedom, a bipartisan independent agency of the US government, has placed Egypt on its watch list of countries that require close monitoring due to the nature and extent of violations of religious freedom engaged in or tolerated by the government. According to a 2010 Pew Global Attitudes survey, 84% of Egyptians polled supported the death penalty for those who leave Islam; 77% supported whippings and cutting off of hands for theft and robbery; and 82% support stoning a person who commits adultery. \n### Question:\nWhat percentage of Egyptians polled support death penalty for those leaving Islam? \n\n\n### Answer:\n84%</s>'

In [ ]:
processedTrainQADS[0]

'### Context:\nThe Pew Forum on Religion & Public Life ranks Egypt as the fifth worst country in the world for religious freedom. The United States Commission on International Religious Freedom, a bipartisan independent agency of the US government, has placed Egypt on its watch list of countries that require close monitoring due to the nature and extent of violations of religious freedom engaged in or tolerated by the government. According to a 2010 Pew Global Attitudes survey, 84% of Egyptians polled supported the death penalty for those who leave Islam; 77% supported whippings and cutting off of hands for theft and robbery; and 82% support stoning a person who commits adultery. \n### Question:\nWhat percentage of Egyptians polled support death penalty for those leaving Islam? \n\n\n### Answer:\n84%</s>'

In [ ]:
len(processedTrainQADS[0])

807

In [ ]:
tokenizedStrIds = qaTokenizer(processedTrainQADS[0], return_tensors='pt')["input_ids"]

In [ ]:
tokenizedStrIds

tensor([[21017, 30532,    25,   198,   464, 21805, 14867,   319, 21741,  1222,
          5094,  5155,  9803,  6365,   355,   262,  8150,  5290,  1499,   287,
           262,   995,   329,  4158,  4925,    13,   383,  1578,  1829,  4513,
           319,  4037, 23945, 10204,    11,   257, 20953,  4795,  4086,   286,
           262,  1294,  1230,    11,   468,  4624,  6365,   319,   663,  2342,
          1351,   286,  2678,   326,  2421,  1969,  9904,  2233,   284,   262,
          3450,   290,  6287,   286, 11734,   286,  4158,  4925,  7953,   287,
           393, 28197,   416,   262,  1230,    13,  4784,   284,   257,  3050,
         21805,  8060,  3460, 10455,  5526,    11,  9508,     4,   286, 32658,
         37698,  4855,   262,  1918,  7389,   329,   883,   508,  2666,  3449,
            26,  8541,     4,  4855,   348,  3974,   654,   290,  7720,   572,
           286,  2832,   329, 12402,   290, 18609,    26,   290,  9415,     4,
          1104,   336, 12484,   257,  1048,   508, 2

In [ ]:
qaTokenizer.decode(tokenizedStrIds[0])

'### Context:\nThe Pew Forum on Religion & Public Life ranks Egypt as the fifth worst country in the world for religious freedom. The United States Commission on International Religious Freedom, a bipartisan independent agency of the US government, has placed Egypt on its watch list of countries that require close monitoring due to the nature and extent of violations of religious freedom engaged in or tolerated by the government. According to a 2010 Pew Global Attitudes survey, 84% of Egyptians polled supported the death penalty for those who leave Islam; 77% supported whippings and cutting off of hands for theft and robbery; and 82% support stoning a person who commits adultery. \n### Question:\nWhat percentage of Egyptians polled support death penalty for those leaving Islam? \n\n\n### Answer:\n84%</s>'

In [ ]:
qaTokenizer("\n", return_tensors="pt")

{'input_ids': tensor([[198]]), 'attention_mask': tensor([[1]])}

In [ ]:
collator.torch_call(qaTokenizer(processedTrainQADS[0], return_tensors='pt')["input_ids"])

{'input_ids': tensor([[21017, 30532,    25,   198,   464, 21805, 14867,   319, 21741,  1222,
           5094,  5155,  9803,  6365,   355,   262,  8150,  5290,  1499,   287,
            262,   995,   329,  4158,  4925,    13,   383,  1578,  1829,  4513,
            319,  4037, 23945, 10204,    11,   257, 20953,  4795,  4086,   286,
            262,  1294,  1230,    11,   468,  4624,  6365,   319,   663,  2342,
           1351,   286,  2678,   326,  2421,  1969,  9904,  2233,   284,   262,
           3450,   290,  6287,   286, 11734,   286,  4158,  4925,  7953,   287,
            393, 28197,   416,   262,  1230,    13,  4784,   284,   257,  3050,
          21805,  8060,  3460, 10455,  5526,    11,  9508,     4,   286, 32658,
          37698,  4855,   262,  1918,  7389,   329,   883,   508,  2666,  3449,
             26,  8541,     4,  4855,   348,  3974,   654,   290,  7720,   572,
            286,  2832,   329, 12402,   290, 18609,    26,   290,  9415,     4,
           1104,   336, 124

In [ ]:
processedTrainQADS[0]

'### Context:\nThe Pew Forum on Religion & Public Life ranks Egypt as the fifth worst country in the world for religious freedom. The United States Commission on International Religious Freedom, a bipartisan independent agency of the US government, has placed Egypt on its watch list of countries that require close monitoring due to the nature and extent of violations of religious freedom engaged in or tolerated by the government. According to a 2010 Pew Global Attitudes survey, 84% of Egyptians polled supported the death penalty for those who leave Islam; 77% supported whippings and cutting off of hands for theft and robbery; and 82% support stoning a person who commits adultery. \n### Question:\nWhat percentage of Egyptians polled support death penalty for those leaving Islam? \n\n\n### Answer:\n84%</s>'

<details>
    <summary>Click here for the solution</summary>

```python
model_name = "EleutherAI/gpt-neo-125m"

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
```

</details>


In [ ]:
qaModel.base_model

GPTNeoModel(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(2048, 768)
  (drop): Dropout(p=0.0, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPTNeoBlock(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPTNeoAttention(
        (attention): GPTNeoSelfAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=768, out_features=768, bias=False)
          (v_proj): Linear(in_features=768, out_features=768, bias=False)
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPTNeoMLP(
        (c_fc): Linear(in_features=768, out_features=3072, bias=True)
        (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        (act): NewGELUActivation()

In [ ]:
lora_config = LoraConfig(
    r=16,  # Low-rank dimension
    lora_alpha=32,  # Scaling factor
    target_modules=["k_proj", "q_proj", "v_proj"],  # Modules to apply LoRA
    lora_dropout=0.1,  # Dropout rate
    task_type=TaskType.CAUSAL_LM  # Task type should be causal language model
)

loraConfig = LoraConfig(r = 16,
                        lora_alpha = 32,
                        target_modules = ["q_proj", "v_proj", "k_proj"],
                        lora_dropout = 0.1,
                        task_type = TaskType.CAUSAL_LM,
                        bias = "none",
                        )

loraQAModel = get_peft_model(qaModel, lora_config)

In [ ]:
loraQAModel

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoForCausalLM(
      (transformer): GPTNeoModel(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(2048, 768)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPTNeoBlock(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPTNeoAttention(
              (attention): GPTNeoSelfAttention(
                (attn_dropout): Dropout(p=0.0, inplace=False)
                (resid_dropout): Dropout(p=0.0, inplace=False)
                (k_proj): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=16, bias=False)
                  )
                  (lora_B

In [ ]:
def tokenizeDatasetItem(dsItem):
    return qaTokenizer(dsItem, return_tensors="pt", max_length=1024, truncation=True, padding=True)

In [ ]:
tokenizedTrainQADS = ListDataset([tokenizeDatasetItem(x) for x in processedTrainQADS.original_list])

In [ ]:
tokenizedTestQADS = ListDataset([tokenizeDatasetItem(x) for x in processedTestQADS.original_list])

In [ ]:
tokenizedTrainQADS[0]

{'input_ids': tensor([[21017, 30532,    25,   198,   464, 21805, 14867,   319, 21741,  1222,
          5094,  5155,  9803,  6365,   355,   262,  8150,  5290,  1499,   287,
           262,   995,   329,  4158,  4925,    13,   383,  1578,  1829,  4513,
           319,  4037, 23945, 10204,    11,   257, 20953,  4795,  4086,   286,
           262,  1294,  1230,    11,   468,  4624,  6365,   319,   663,  2342,
          1351,   286,  2678,   326,  2421,  1969,  9904,  2233,   284,   262,
          3450,   290,  6287,   286, 11734,   286,  4158,  4925,  7953,   287,
           393, 28197,   416,   262,  1230,    13,  4784,   284,   257,  3050,
         21805,  8060,  3460, 10455,  5526,    11,  9508,     4,   286, 32658,
         37698,  4855,   262,  1918,  7389,   329,   883,   508,  2666,  3449,
            26,  8541,     4,  4855,   348,  3974,   654,   290,  7720,   572,
           286,  2832,   329, 12402,   290, 18609,    26,   290,  9415,     4,
          1104,   336, 12484,   257,  

In [ ]:
qaCustomTrainer = SFTTrainer(
    loraQAModel,
    tokenizer=qaTokenizer,
    train_dataset = trainQADS,
    eval_dataset = testQADS,
    formatting_func = formattQaDSPrompt,
    args = qatrainingArgs,
    #packing=False,
    data_collator=collator,
)

Map:   0%|          | 0/87132 [00:00<?, ? examples/s]

Map:   0%|          | 0/10399 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
qaCustomTrainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: marcin-wizgird (gatech-omcs-research) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,1.549800
1000,0.699200
1500,0.643500
2000,0.596200
2500,0.579200
3000,0.559000
3500,0.551400
4000,0.546200
4500,0.542800
5000,0.529700


Step,Training Loss
500,1.549800
1000,0.699200
1500,0.643500
2000,0.596200
2500,0.579200
3000,0.559000
3500,0.551400
4000,0.546200
4500,0.542800
5000,0.529700


TrainOutput(global_step=54460, training_loss=0.4513186328340181, metrics={'train_runtime': 6738.8264, 'train_samples_per_second': 129.298, 'train_steps_per_second': 8.082, 'total_flos': 1.4637863960705434e+17, 'train_loss': 0.4513186328340181, 'epoch': 10.0})